In [1]:
import pyspark
from pyspark.sql.functions import col,when,size,min, max,length
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master('local[*]')\
    .appName('topic_modelling')\
    .getOrCreate()

In [3]:
df_parquet = spark.read.parquet("part-00000-a8a717e8-bf83-42e3-a0b7-084971053d99-c000.snappy.parquet")

In [4]:
# create a column and put in the detected language code
from langdetect import detect
getCode = lambda x:detect(x)
sent = "this is eng"
getCode(sent)

'en'

In [5]:
# create a column and put in the detected language code
from langdetect import detect
# app_name_result = df_parquet.select(col('description')).collect()
# # create udf function to add a new column
# import pyspark.sql.functions as F
# from pyspark.sql.types import *

# def detect_lang(value):
#   return detect(value)

# #convert to a UDF Function by passing in the function and return type of function
# udf_detect_lang = F.udf(detect_lang, StringType())
# df_with_code = df_parquet.withColumn('lang_code', udf_detect_lang(df_parquet.description))
# df_with_code.show(5)

# easier method to convert to pandas then apply then function
df_parquet_pd = df_parquet.toPandas()
df_parquet_pd['lang_code'] = df_parquet_pd['description'].apply(getCode)
df_parquet_pd


,app_name,bundle,description,genres,os_platform,lang_code
0,Black Box - Movie Listing,958957112,Black Box Movie Listing App is a collection li...,"[Entertainment, Magazines & Newspapers]",IOS,en
1,Knife Throwing Max,1491396285,you will have fun in this game\n100+ challengi...,"[Games_Sports, Games_Role Playing]",IOS,en
2,爆料公社,1387765782,全台最大社群「爆料公社」APP 上線啦，我們把社群解放了，不只有勁爆的爆料內容，還貼心的加上...,[Social Networking],IOS,zh-tw
3,Lidow - Photo Editor & Collage,894532288,layout/grid/collage、Square/no crop for instagr...,[Photo & Video],IOS,en
4,Vidstitch Frames for Instagram,712908978,"■ #Vidstitch\n■ Featured on 148apps, Stelapps,...","[Photo & Video, Social Networking]",IOS,en
...,...,...,...,...,...,...
860,Barber Hair Salon & Beard Makeover,com.hmg.haircutgames.beardsalon.barberhaircut,Barber hair makeover salon game is for those w...,"[ENTERTAINMENT, FAMILY_PRETEND]",ANDROID,en
861,DJ Remix Offline 2020,com.mpro.djremixoffline2020,Dj Remix Offline 2020 Mp3 merupakan aplikasi m...,[MUSIC_AND_AUDIO],ANDROID,id
862,Sahih Bukhari (English),com.bangladroid.sahihbukhari,We have made this application from the book of...,[BOOKS_AND_REFERENCE],ANDROID,en
863,Two Minute English,com.astrobix.twominuteenglish,Improve your spoken English rapidly with the h...,[EDUCATION],ANDROID,en


In [14]:
import pandas as pd
code_ref = pd.read_excel('lang_code.xlsx')
df_parquet_pd = pd.merge(df_parquet_pd,code_ref,how = 'left',on = 'lang_code')

In [15]:
df_parquet_pd

,app_name,bundle,description,genres,os_platform,lang_code,language_name
0,Black Box - Movie Listing,958957112,Black Box Movie Listing App is a collection li...,"[Entertainment, Magazines & Newspapers]",IOS,en,English
1,Knife Throwing Max,1491396285,you will have fun in this game\n100+ challengi...,"[Games_Sports, Games_Role Playing]",IOS,en,English
2,爆料公社,1387765782,全台最大社群「爆料公社」APP 上線啦，我們把社群解放了，不只有勁爆的爆料內容，還貼心的加上...,[Social Networking],IOS,zh-tw,Chinese
3,Lidow - Photo Editor & Collage,894532288,layout/grid/collage、Square/no crop for instagr...,[Photo & Video],IOS,en,English
4,Vidstitch Frames for Instagram,712908978,"■ #Vidstitch\n■ Featured on 148apps, Stelapps,...","[Photo & Video, Social Networking]",IOS,en,English
...,...,...,...,...,...,...,...
860,Barber Hair Salon & Beard Makeover,com.hmg.haircutgames.beardsalon.barberhaircut,Barber hair makeover salon game is for those w...,"[ENTERTAINMENT, FAMILY_PRETEND]",ANDROID,en,English
861,DJ Remix Offline 2020,com.mpro.djremixoffline2020,Dj Remix Offline 2020 Mp3 merupakan aplikasi m...,[MUSIC_AND_AUDIO],ANDROID,id,Indonesian
862,Sahih Bukhari (English),com.bangladroid.sahihbukhari,We have made this application from the book of...,[BOOKS_AND_REFERENCE],ANDROID,en,English
863,Two Minute English,com.astrobix.twominuteenglish,Improve your spoken English rapidly with the h...,[EDUCATION],ANDROID,en,English


In [16]:
# load indonesian data
df_parquet_id = df_parquet_pd.loc[df_parquet_pd['lang_code']=='id']
df_parquet_id

,app_name,bundle,description,genres,os_platform,lang_code,language_name
12,Cara Menggunakan Jam Imoo Anak,com.jamimoo.guide,Imoo Watch Phone hadir dengan berbagai fitur m...,[BOOKS_AND_REFERENCE],ANDROID,id,Indonesian
20,Lagu Dangdut Hamdan ATT Offline,com.zakiy.hamdan,Aplikasi Ini Bersifat Offline Sehingga Tidak M...,[MUSIC_AND_AUDIO],ANDROID,id,Indonesian
28,Surah Al Baqarah Mp3 Terjemah,com.andromo.dev572857.app594683,Aplikasi ini berisi kumpulan audio surah Al Ba...,[MUSIC_AND_AUDIO],ANDROID,id,Indonesian
34,Kitab Kuning Fathul Qorib,com.guidestudio.kitabfathulqorib,"Kitab kuning, dalam pendidikan agama islam, me...",[BOOKS_AND_REFERENCE],ANDROID,id,Indonesian
38,Kumpulan Lagu Jamrud dan Lagu Padi offline,com.imhsfh.jamrudpadi,kumpulan lagu jamrud dan padi lengkap disertai...,[MUSIC_AND_AUDIO],ANDROID,id,Indonesian
...,...,...,...,...,...,...,...
822,Lagu Orkes Gambus Modern,com.dragobogy.orkesgambusmodern,Kumpulan Lagu Orkes Gambus Modern Mp3 adalah a...,[MUSIC_AND_AUDIO],ANDROID,id,Indonesian
838,Koleksi Soal Jawab Ustaz Azhar Idrus - Terbaru,sehaladesign.uai,Koleksi Soal Jawab Ustaz Azhar Idrus (UAI) vid...,[BOOKS_AND_REFERENCE],ANDROID,id,Indonesian
842,O'zbek - Rusda Tarjimon,com.linguaapps.translator.uzbek.ru,O'zbek - Rusda Tarjimon dasturi - bepul va ul...,[BOOKS_AND_REFERENCE],ANDROID,id,Indonesian
858,Lagu Galau Offline 2020,com.kermaxdevteam.lagugalaupatahhati,Kumpulan lagu galau patah hati terbaru 2020 pa...,[MUSIC_AND_AUDIO],ANDROID,id,Indonesian


In [17]:
df_spark = spark.createDataFrame(df_parquet_en)

In [26]:
# Step 1. Text cleasing with punctuations
from pyspark.sql.functions import regexp_replace
REGEX = '[()`_,.?!?@#$%^&*+\/\d]'
df_spark = df_spark.withColumn("description_clean",regexp_replace(df_spark.description,REGEX,' '))
df_spark.select("description_clean").show(truncate= False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [28]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stemF = lambda x: stemmer.stem(x)

In [29]:
from pyspark.sql.types import *
import pyspark.sql.functions as F

udf_stemF = F.udf(stemF,StringType())
df_spark = df_spark.withColumn("description_stem",udf_stemF(df_spark.description_clean))
df_spark.columns

['app_name',
 'bundle',
 'description',
 'genres',
 'os_platform',
 'lang_code',
 'description_clean',
 'description_token',
 'description_stem']

In [31]:
df_spark.select("description_stem").show(truncate = False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [33]:
# Step 2. Tokenization
df_spark = df_spark.drop("description_token")
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol='description_stem',outputCol='description_token')
df_spark = tokenizer.transform(df_spark)
df_spark.show()

+--------------------+--------------------+--------------------+--------------------+-----------+---------+--------------------+--------------------+--------------------+
|            app_name|              bundle|         description|              genres|os_platform|lang_code|   description_clean|    description_stem|   description_token|
+--------------------+--------------------+--------------------+--------------------+-----------+---------+--------------------+--------------------+--------------------+
|Cara Menggunakan ...|   com.jamimoo.guide|Imoo Watch Phone ...|[BOOKS_AND_REFERE...|    ANDROID|       id|Imoo Watch Phone ...|imoo watch phone ...|[imoo, watch, pho...|
|Lagu Dangdut Hamd...|    com.zakiy.hamdan|Aplikasi Ini Bers...|   [MUSIC_AND_AUDIO]|    ANDROID|       id|Aplikasi Ini Bers...|aplikasi ini sifa...|[aplikasi, ini, s...|
|Surah Al Baqarah ...|com.andromo.dev57...|Aplikasi ini beri...|   [MUSIC_AND_AUDIO]|    ANDROID|       id|Aplikasi ini beri...|aplikasi ini isi 

In [55]:
df_spark.select("description","description_clean","description_token").show(truncate = False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [45]:
# Step 3. prepare stopword list
stopword_id = pd.read_csv("stopword_id.csv",header = None,names=(['stopword']))

In [62]:
stopword_list = [x for x in stopword_id['stopword'].values]

In [53]:
print(type(stopword_list),len(stopword_list))

<class 'list'> 758


In [58]:
df_pd_desc_final = df_spark.toPandas()

In [59]:
df_pd_desc_final

,app_name,bundle,description,genres,os_platform,lang_code,description_clean,description_stem,description_token
0,Cara Menggunakan Jam Imoo Anak,com.jamimoo.guide,Imoo Watch Phone hadir dengan berbagai fitur m...,[BOOKS_AND_REFERENCE],ANDROID,id,Imoo Watch Phone hadir dengan berbagai fitur m...,imoo watch phone hadir dengan bagai fitur tari...,"[imoo, watch, phone, hadir, dengan, bagai, fit..."
1,Lagu Dangdut Hamdan ATT Offline,com.zakiy.hamdan,Aplikasi Ini Bersifat Offline Sehingga Tidak M...,[MUSIC_AND_AUDIO],ANDROID,id,Aplikasi Ini Bersifat Offline Sehingga Tidak M...,aplikasi ini sifat offline sehingga tidak maka...,"[aplikasi, ini, sifat, offline, sehingga, tida..."
2,Surah Al Baqarah Mp3 Terjemah,com.andromo.dev572857.app594683,Aplikasi ini berisi kumpulan audio surah Al Ba...,[MUSIC_AND_AUDIO],ANDROID,id,Aplikasi ini berisi kumpulan audio surah Al Ba...,aplikasi ini isi kumpul audio surah al baqarah...,"[aplikasi, ini, isi, kumpul, audio, surah, al,..."
3,Kitab Kuning Fathul Qorib,com.guidestudio.kitabfathulqorib,"Kitab kuning, dalam pendidikan agama islam, me...",[BOOKS_AND_REFERENCE],ANDROID,id,Kitab kuning dalam pendidikan agama islam me...,kitab kuning dalam didik agama islam rujuk kep...,"[kitab, kuning, dalam, didik, agama, islam, ru..."
4,Kumpulan Lagu Jamrud dan Lagu Padi offline,com.imhsfh.jamrudpadi,kumpulan lagu jamrud dan padi lengkap disertai...,[MUSIC_AND_AUDIO],ANDROID,id,kumpulan lagu jamrud dan padi lengkap disertai...,kumpul lagu jamrud dan padi lengkap serta liri...,"[kumpul, lagu, jamrud, dan, padi, lengkap, ser..."
...,...,...,...,...,...,...,...,...,...
77,Lagu Orkes Gambus Modern,com.dragobogy.orkesgambusmodern,Kumpulan Lagu Orkes Gambus Modern Mp3 adalah a...,[MUSIC_AND_AUDIO],ANDROID,id,Kumpulan Lagu Orkes Gambus Modern Mp adalah a...,kumpul lagu orkes gambus modern mp adalah apli...,"[kumpul, lagu, orkes, gambus, modern, mp, adal..."
78,Koleksi Soal Jawab Ustaz Azhar Idrus - Terbaru,sehaladesign.uai,Koleksi Soal Jawab Ustaz Azhar Idrus (UAI) vid...,[BOOKS_AND_REFERENCE],ANDROID,id,Koleksi Soal Jawab Ustaz Azhar Idrus UAI vid...,koleksi soal jawab ustaz azhar idrus uai video...,"[koleksi, soal, jawab, ustaz, azhar, idrus, ua..."
79,O'zbek - Rusda Tarjimon,com.linguaapps.translator.uzbek.ru,O'zbek - Rusda Tarjimon dasturi - bepul va ul...,[BOOKS_AND_REFERENCE],ANDROID,id,O'zbek - Rusda Tarjimon dasturi - bepul va ul...,o zbek - rusda tarjimon dasturi - bepul va ula...,"[o, zbek, -, rusda, tarjimon, dasturi, -, bepu..."
80,Lagu Galau Offline 2020,com.kermaxdevteam.lagugalaupatahhati,Kumpulan lagu galau patah hati terbaru 2020 pa...,[MUSIC_AND_AUDIO],ANDROID,id,Kumpulan lagu galau patah hati terbaru pa...,kumpul lagu galau patah hati baru paling popul...,"[kumpul, lagu, galau, patah, hati, baru, palin..."


### Note: IDF vector must be trained with large corpus, otherwise lose the advance of IDF

In [60]:
# get the "description" column
joinF= lambda x:" ".join(x)
df_pd_desc_final["description_join"] = df_pd_desc_final["description_token"].apply(joinF)

In [ ]:
# corpus_list = df_pd_desc_final["description_join"].tolist()

In [63]:
# Step 4. TF-IDF countvector, Transform documents to document-term matrix
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df=0.95,stop_words = stopword_list) #ignore terms that have a document frequency strictly higher than the given threshold
dtm = cv.fit_transform(df_pd_desc_final["description_join"])

C:\Users\ChengXiaotian\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [64]:
dtm.shape

(82, 2289)

In [65]:
feature_names = cv.get_feature_names()

In [66]:
feature_names

['aamiin',
 'aan',
 'abad',
 'abadi',
 'abdul',
 'abdullah',
 'abi',
 'abraham',
 'abu',
 'aburizal',
 'acak',
 'aceh',
 'acu',
 'ad',
 'adam',
 'adek',
 'adjective',
 'adjectives',
 'adjeng',
 'adl',
 'admin',
 'ads',
 'adverb',
 'adverbs',
 'adzan',
 'affiliated',
 'afiliasi',
 'afrikans',
 'afu',
 'agam',
 'agama',
 'agatha',
 'agung',
 'agustus',
 'ahli',
 'ahmad',
 'aidilfitri',
 'ainun',
 'air',
 'aja',
 'ajak',
 'ajar',
 'aji',
 'akademik',
 'akhirat',
 'akhlaq',
 'akibat',
 'akik',
 'aks',
 'akses',
 'aktif',
 'aktivitas',
 'akurasi',
 'al',
 'alaihi',
 'alam',
 'alami',
 'alarm',
 'alas',
 'alat',
 'albania',
 'albert',
 'albino',
 'album',
 'alfiyah',
 'ali',
 'alias',
 'alim',
 'alin',
 'alit',
 'aliyah',
 'all',
 'allah',
 'alon',
 'alquran',
 'also',
 'alun',
 'alva',
 'always',
 'am',
 'ama',
 'amal',
 'amalan',
 'aman',
 'amandemen',
 'ambil',
 'ambyar',
 'amhara',
 'amin',
 'ampuh',
 'an',
 'ana',
 'anak',
 'ananta',
 'ancam',
 'and',
 'andai',
 'andrea',
 'andrie',
 'a

In [68]:
cv.vocabulary_

{'imoo': 759,
 'watch': 2229,
 'phone': 1508,
 'hadir': 649,
 'fitur': 549,
 'tarik': 1989,
 'video': 2194,
 'call': 309,
 'sms': 1874,
 'photo': 1509,
 'gps': 626,
 'anak': 92,
 'ngebet': 1366,
 'milik': 1263,
 'harga': 679,
 'nya': 1394,
 'lumayan': 1148,
 'mahal': 1161,
 'jam': 832,
 'barang': 200,
 'tiru': 2075,
 'aplikasi': 115,
 'putus': 1593,
 'beli': 226,
 'goda': 618,
 'murah': 1319,
 'kualitas': 1035,
 'emper': 502,
 'produk': 1567,
 'originalnya': 1426,
 'menu': 1249,
 'utama': 2177,
 'setting': 1823,
 'asli': 144,
 'kw': 1052,
 'daftar': 384,
 'barcode': 203,
 'scanner': 1764,
 'biru': 256,
 'beneran': 231,
 'aktif': 50,
 'edit': 478,
 'foto': 557,
 'film': 542,
 'game': 582,
 'lokasi': 1135,
 'lacak': 1056,
 'lagu': 1060,
 'main': 1166,
 'maps': 1200,
 'nomor': 1377,
 'nelpon': 1357,
 'nonton': 1380,
 'ok': 1408,
 'google': 621,
 'sifat': 1843,
 'offline': 1406,
 'makan': 1171,
 'kuota': 1047,
 'internet': 787,
 'ringtone': 1683,
 'risi': 1686,
 'hamdan': 666,
 'att': 152,

In [69]:
# Step 5. TfidfTransformer to compute the IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidfTransformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidfTransformer.fit(dtm)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [70]:
def get_keyword_list(doc):
    # generate tfidf vector for given doc
    tf_idf_vector = tfidfTransformer.transform(cv.transform([doc]))
    
    # Return a C00rdinate representation of this matrix
    coo_matrix = tf_idf_vector.tocoo()
    # sorted_items = sort_coo(coo_matrix)

    # get Top-n index list
    index_sort = np.argsort(coo_matrix.data)[-topn:]
    
    keyword_list = []
    # extract keywords
    for idx in index_sort[::-1]: 
        keyword_list.append(feature_names[coo_matrix.col[idx]])
    
    return keyword_list

In [72]:
# Step 6. Get consolidated keyword list for entire dataset
import numpy as np

# get topn = 10, keywords
topn = 10
df_pd_desc_final['description_keyword_list']=df_pd_desc_final['description_join'].apply(get_keyword_list)

In [73]:
# output for viewing
df_pd_desc_final['description_keyword_list']

0     [imoo, jam, phone, watch, harga, kw, setting, ...
1     [att, hamdan, makan, online, ringtone, sifat, ...
2     [baqarah, al, surat, rasulullah, saw, baca, su...
3     [kitab, ulama, tulis, kuning, ilmu, karya, isl...
4     [jamrud, padi, lirik, lagu, rakit, indah, kump...
                            ...                        
77    [gambus, modern, lagu, content, orkes, from, o...
78    [azhar, ustaz, idrus, lawak, uai, koleksi, kon...
79    [va, zbek, rusda, mumkin, ushbu, uchun, tarjim...
80    [galau, ng, hati, lagu, konten, cinta, romanti...
81    [remix, dj, lagu, aplikasi, klik, hak, cipta, ...
Name: description_keyword_list, Length: 82, dtype: object

In [74]:
df_desc_final_spark = spark.createDataFrame(df_pd_desc_final)

In [75]:
# create new columns to store keyword
for i in range(topn):
        df_desc_final_spark = df_desc_final_spark.withColumn("keyword-"+str(i+1),df_desc_final_spark['description_keyword_list'][i])

In [76]:
df_desc_final_spark.columns

['app_name',
 'bundle',
 'description',
 'genres',
 'os_platform',
 'lang_code',
 'description_clean',
 'description_stem',
 'description_token',
 'description_join',
 'description_keyword_list',
 'keyword-1',
 'keyword-2',
 'keyword-3',
 'keyword-4',
 'keyword-5',
 'keyword-6',
 'keyword-7',
 'keyword-8',
 'keyword-9',
 'keyword-10']

In [77]:
# save the final keyword spreadsheet to local
df_desc_final = df_desc_final_spark.toPandas()
df_desc_final.to_csv('desc_keyword_id.csv')

In [ ]:
# # BLOCK for testing individual sentence
# # get the document that we want to extract from
# test_idx = 3
# doc_test = df_pd_desc_final['description_join'][test_idx]

# # generate tfidf vector for given doc
# tf_idf_vector = tfidfTransformer.transform(cv.transform([doc_test]))

In [ ]:
# import numpy as np
# # Return a C00rdinate representation of this matrix
# coo_matrix = tf_idf_vector.tocoo()
# # sorted_items = sort_coo(coo_matrix)

# # get Top-n list
# index_sort = np.argsort(coo_matrix.data)[-10:]

# # extract keywords, topn
# print("Top-n keywords in the given document:\n")
# for idx in index_sort[::-1]: 

#     print(feature_names[coo_matrix.col[idx]],":",coo_matrix.data[idx])

# # keywords = extract_topn_from_vector(feature_names,sorted_items,10)

# # # print result
# # print("Top-n keywords in the given document:\n")
# # for k in keywords:
# #     print(k,":",keywords[k])

## -- END --

In [ ]:
# doc_test

In [ ]:
# doc_origin = df_pd_desc_final['description'][test_idx]
# doc_origin

In [ ]:
# df_pd_desc_final['app_name'][test_idx]

In [ ]:
# df_pd_desc_final['genres'][test_idx]

In [ ]:
# dm = tf_idf_vector.todense()

In [ ]:
# def sort_coo(coo_matrix):
#     tuples = zip(coo_matrix.col,coo_matrix.data)
#     return sorted(tuples,key=lambda x:x[1],reverse=True)

# def extract_topn_from_vector(feature_names,sorted_items,topn):
#     """get the feature names according to the tf-idf score of top-n items"""
    
#     #use only top-n itmes
#     sorted_items = sorted_items[:topn]
    
#     score_vals = []
#     features_vals = []
    
#     # word index and corresponding tf-idf score
#     for index,score in sorted_items:
        
#         #feature name and score
#         score_vals.append(round(score,3))
#         features_vals.append(feature_names[index])
        
#     # create a tuples of feature,score
#     results = {}
#     for index in range(len(features_vals)):
#         results[features_vals[index]] = score_vals[index]
        
#     return results